# **Importação e Visualização dos Dados**

In [ ]:
# !pip install --upgrade huggingface_hub
# !pip install datasets

In [ ]:
from datasets import load_dataset
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import io

FIG_NUMBER = 1
IMG_HEIGHT = 512
IMG_WIDTH = 512

In [ ]:
%%time
dataset = load_dataset("pierreguillou/DocLayNet-base", trust_remote_code=True)
# dataset = load_dataset("pierreguillou/DocLayNet-small", trust_remote_code=True)

In [ ]:
df_train = dataset['train'].to_pandas()
df_validation = dataset['validation'].to_pandas()
df_test = dataset['test'].to_pandas()

## **Distribuição dos Dados**

In [ ]:
doc_categories = df_train.doc_category.unique()
num_classes = len(doc_categories)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(20, 6))

def plot_image_distribution(dataframe, axis, base_name, color):
    global FIG_NUMBER
    num_images = []
    for i in range(num_classes):
        class_data = dataframe[dataframe['doc_category'] == doc_categories[i]]
        count = class_data.shape[0]
        num_images.append(count)

    axs[axis].bar(doc_categories, num_images, color=f'{color}')
    axs[axis].set_xticks(range(len(doc_categories)))
    axs[axis].set_xticklabels(doc_categories, rotation=45)
    axs[axis].set_title(f'Figura {FIG_NUMBER} - Base de {base_name}')
    axs[axis].set_xlabel('Categorias')
    axs[axis].set_ylabel('Imagens')
    FIG_NUMBER += 1

plot_image_distribution(df_train, 0, 'Treino', 'lightgreen')
plot_image_distribution(df_validation, 1, 'Validação', 'mediumseagreen')
plot_image_distribution(df_test, 2, 'Teste', 'darkgreen')

plt.tight_layout()
plt.show()

In [ ]:
class_data = df_train[df_train['doc_category'] == 'manuals']
print(class_data.shape[0])

class_data = df_validation[df_validation['doc_category'] == 'manuals']
print(class_data.shape[0])

class_data = df_test[df_test['doc_category'] == 'manuals']
print(class_data.shape[0])

> - A distribuição das categorias de imagens é relativamente semelhante entre as bases, a categoria com mais disparidade é ***manuals***, sendo:  
> **1422** para a base de **treino**  
> **180** para a base de **validação**  
> **77** para a base de **teste**  

## **Pré Processamento da Imagem**

In [ ]:
print(df_train['image'][0]['bytes'])
# array = np.array(df_train['image'][0]['bytes'])
# array = np.frombuffer(df_train['image'][0]['bytes'])
# array

In [ ]:
def get_png_array(df_row):
    png_bytes = df_row['bytes']

    # Converta os bytes da imagem em um objeto Image
    image = Image.open(io.BytesIO(png_bytes))
    
    grayscale_image = image.convert('L')
    
    # Redimensione a imagem
    resized_image = grayscale_image.resize((IMG_WIDTH, IMG_HEIGHT))

    # Converta a imagem redimensionada para um array NumPy
    image_array = np.array(resized_image)
    
    return image_array

In [ ]:
df_train['image_array'] = df_train['image'].apply(lambda x: get_png_array(x))

In [ ]:
df_validation['image_array'] = df_validation['image'].apply(lambda x: get_png_array(x))

## **Implementação da Rede Neural Artificial**

In [ ]:
# x_train = df_train['image_array']
# y_train = df_train['doc_category']
x_train = np.stack(df_train['image_array'].values).astype(np.float32) / 255.0
x_val = np.stack(df_validation['image_array'].values).astype(np.float32) / 255.0

In [ ]:
x_train = x_train.reshape((x_train.shape[0], IMG_WIDTH * IMG_HEIGHT))
x_val = x_val.reshape((x_val.shape[0], IMG_WIDTH * IMG_HEIGHT))

In [ ]:
y_train = df_train['doc_category']
y_val = df_validation['doc_category']

In [ ]:
# !pip install -U scikit-learn

In [ ]:
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer()

y_train_encoded = label_binarizer.fit_transform(y_train)
y_val_encoded = label_binarizer.fit_transform(y_val)

### **Rede Neural Artificial Densa**

In [ ]:
# !pip install --upgrade keras
# !pip install tensorflow

In [ ]:
# Importando bibliotecas para criação e treinamento dos modelos
import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from keras.regularizers import l2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
# Criando tf.data.Dataset
batch_size = 500

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train_encoded))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val_encoded))
val_dataset = val_dataset.batch(batch_size).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
neural_network_modelv2 = Sequential([
    Flatten(input_shape=(IMG_HEIGHT*IMG_WIDTH,)),
    Dropout(0.2),
    Dense(200, activation='relu'),
    Dropout(0.2),
    Dense(100, activation='relu'),
    Dropout(0.2),
    Dense(50, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

adam = Adam(learning_rate=0.001)
neural_network_modelv2.compile(optimizer=adam,
              loss=["categorical_crossentropy"],
              metrics=['accuracy'])

neural_network_modelv2.summary()

In [ ]:
callback = EarlyStopping(monitor='loss', patience=5)
neural_network_historyv2 = neural_network_modelv2.fit(
    train_dataset,
    epochs=100,
    # verbose=0,
    validation_data=val_dataset,
    callbacks=[callback]
)

In [ ]:
""" callback = EarlyStopping(monitor='loss', patience=5)
neural_network_historyv2 = neural_network_modelv2.fit(np.array(x_train),
          np.array(y_train_encoded),
          epochs=100,
          batch_size=500,
          verbose=0,
          shuffle=True,
          validation_data=(np.array(x_val), np.array(y_val_encoded)),
          callbacks=[callback]) """

In [ ]:
""" fig, axs = plt.subplots(1,2,figsize=(10,8))
axs[1, 0].plot(neural_network_historyv2.history['accuracy'], color='violet', label='accuracy')
axs[1, 0].plot(neural_network_historyv2.history['val_accuracy'], color='darkviolet', label='val_accuracy')
axs[1, 0].set_title('Acurácia Rede Neural v2', fontsize=10)

axs[1, 1].plot(neural_network_historyv2.history['loss'], color='orange',label='loss')
axs[1, 1].plot(neural_network_historyv2.history['val_loss'], color='tomato', label='val_loss')
axs[1, 1].set_title('Perda Rede Neural v2', fontsize=10)

axs[1, 0].legend(loc="upper left")
axs[1, 1].legend(loc="upper left")
plt.tight_layout()
plt.show() """